## Word Embedding
---


In [1]:
# Import
from nltk.tokenize import word_tokenize
from gensim.models.phrases import Phrases, Phraser
import fuzzysearch
import fuzzywuzzy
import pandas as pd
import PyPDF2

pdfFileObj = open('20221118-FULL.pdf', 'rb') #Create pdf object
pdfReader = PyPDF2.PdfFileReader(pdfFileObj) 

In [2]:
allText = ""
for i in range(100): #Iterating over n pages 
    pageObj = pdfReader.getPage(i)
    page = pageObj.extractText()
    allText += page #Store page content into string variable

allText;

#### Split Text into persons

---

In [3]:
# Preprocessing
allText = allText.replace(';','')

alltext_string_all = allText.replace('Legal basis',';Legal basis').replace('Identity information',';Identity information').replace('Programme', ';Programme').replace('Birth information',';Birth information').replace('Citizenship information',';Citizenship information').replace('Contact information',';Contact information').replace('Remark',';Remark')
testListSplit = alltext_string_all.split('reference number: EU')
alltext_string_all[:1000]


'EUROPEAN COMMISSION\nFinancial Stability, Financial Services and Capital Markets Union (DG FISMA)\nEuropean Union\nConsolidated Financial Sanctions List\nThis list has been updated on 18/11/2022 11:37European Union Consolidated Financial Sanctions List\nTable of contents\n1. Introduction3 \n2. Individuals or persons3 \n3. Entities or groups631 \n4. Disclaimer792 \nPage 2 on 792European Union Consolidated Financial Sanctions List\n1. INTRODUCTION\nThe present document contains the Consolidated List of persons, groups and entities subject, under EU Sanctions, to an asset freeze and the prohibition to\nmake funds and economic resources available to them. The latest version of this file is here.\n2. INDIVIDUALS OR PERSONS\nEU reference number: EU.1787.1\n;Legal basis: 2017/404 (OJ L63)\n;Programme: AFG - Afghanistan\n;Identity information:\n• Name/Alias: Sirajuddin Jallaloudine Haqqani  Function: Na’ib Amir (Deputy Commander)  \n• Name/Alias: Khalifa  \n• Name/Alias: Saraj Haqani  \n• Nam

In [4]:
# Create dataframe
dbn_df = pd.DataFrame([sub.split(";") for sub in testListSplit])
dbn_df.fillna("",inplace=True)
dbn_df.columns = ['EU Reference', 'Legal basis', 'Programme', 'Identity information', 'Birth information', 'Citizenship information', 'Contact information', 'Remark','','','','','','','','',]



In [7]:
dbn_df

,EU Reference,Legal basis,Programme,Identity information,Birth information,Citizenship information,Contact information,Remark,,,,,,,,
0,"EUROPEAN COMMISSION\nFinancial Stability, Fina...",,,,,,,,,,,,,,,
1,.1787.1\n,Legal basis: 2017/404 (OJ L63)\n,Programme: AFG - Afghanistan\n,Identity information:\n• Name/Alias: Sirajuddi...,Birth information:\n• Birth date: Circa from 1...,Citizenship information:\n• Citizenship: Afgha...,"Contact information: \n• Address: Pakistan, Mi...",Remark: Heading the Haqqani Network as of late...,,,,,,,,
2,.2272.60\n,Legal basis: 2017/404 (OJ L63)\n,Programme: AFG - Afghanistan\n,Identity information:\n• Name/Alias: Nasiruddi...,Birth information:\n• Birth date: Circa from 1...,"Remark: Neka District, Paktika Province, Afgha...",Citizenship information:\n• Citizenship: Afgha...,Contact information: \n• Address: Pakistan \n,Remark: A leader of the Haqqani Network (TAe.0...,,,,,,,
3,.2274.87\n,Legal basis: 2017/404 (OJ L63)\n,Programme: AFG - Afghanistan\n,Identity information:\n• Name/Alias: Gul Agha ...,Birth information:\n• Birth date: Circa 1972 ...,Contact information: \n• Address: Pakistan \n...,Remark: Belongs to Ishaqzai tribe. INTERPOL-UN...,,,,,,,,,
4,.2291.0\n,Legal basis: 2017/404 (OJ L63)\n,Programme: AFG - Afghanistan\n,Identity information:\n• Name/Alias: Amir Abdu...,Birth information:\n• Birth date: Circa 1972 ...,Citizenship information:\n• Citizenship: Afgha...,"Contact information: \n• Address: Pakistan, Ka...",Remark: Believed to be in Afghanistan/Pakistan...,,,,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
383,.3631.49\n,Legal basis: 2017/396 (OJ L60)\n,Programme: COD - Democractic Republic of Congo\n,Identity information:\n• Name/Alias: Mathieu C...,Birth information:\n• Birth date: 08/10/1970 ...,"Remark: Place of birth: Bunia, Ituri\nProvince...",Citizenship information:\n• Citizenship: Congo...,"Contact information: \n• Address: Congo, Democ...",Remark: Mathieu Chui Ngudjolo was the Chief of...,,,,,,,
384,.3632.14\n,Legal basis: 2017/396 (OJ L60)\n,Programme: COD - Democractic Republic of Congo...,Identity information:\n• Name/Alias: Floribert...,Birth information:\n• Birth date: 23/05/1971 ...,Citizenship information:\n• Citizenship: Congo...,Remark: President of FNI\nEU,,,,,,,,,
385,.3633.76\n,Legal basis: 2017/396 (OJ L60)\n,Programme: COD - Democractic Republic of Congo\n,Identity information:\n• Name/Alias: 'Chairman...,Birth information:\n• Birth date: 06/02/1967 ...,Citizenship information:\n• Citizenship: Congo...,,,,,,,,,,
386,.3634.41\n,Legal basis: 2017/396 (OJ L60)\n,Programme: COD - Democractic Republic of Congo\n,Identity information:\n• Name/Alias: Fred Irak...,"Birth information:\n• Birth place: Rwanda, Kin...",Citizenship information:\n• Citizenship: Rwanda\n,"Contact information: \n• Address: Congo, Democ...",Remark: As of June 2016\nEU,,,,,,,,


## Dataframe Clenup 
---

In [10]:
def cleanupDataframe(dataframe):
    dataframe['Legal basis'] = dataframe['Legal basis'].apply(lambda x: x.replace('Legal basis: ','').replace('\n',''))
    dataframe['Programme'] = dataframe['Programme'].apply(lambda x: x.replace('Programme: ','').replace('\n',''))
    dataframe['Identity information'] = dataframe['Identity information'].apply(lambda x: x.replace('Identity information:','').replace('\n','').replace('Name/Alias:',';').replace('•','').replace('Function:',''))
    
    
    return dataframe

def stripDataframe(dataframe):
    dataframe['Identity information'] = dataframe['Identity information'].apply(lambda x: x.strip())
    
    return dataframe

def listCreation(dataframe):
    identityInformation = []
    for i in dataframe:
        temp = dataframe.tolist()
        identityInformation.append(temp)
    return identityInformation


In [11]:
cleanupDataframe(dbn_df)
stripDataframe(dbn_df)


,EU Reference,Legal basis,Programme,Identity information,Birth information,Citizenship information,Contact information,Remark,,,,,,,,
0,"EUROPEAN COMMISSION\nFinancial Stability, Fina...",,,,,,,,,,,,,,,
1,.1787.1\n,2017/404 (OJ L63),AFG - Afghanistan,; Sirajuddin Jallaloudine Haqqani Na’ib Amir...,Birth information:\n• Birth date: Circa from 1...,Citizenship information:\n• Citizenship: Afgha...,"Contact information: \n• Address: Pakistan, Mi...",Remark: Heading the Haqqani Network as of late...,,,,,,,,
2,.2272.60\n,2017/404 (OJ L63),AFG - Afghanistan,; Nasiruddin Haqqani A leader of the Haqqani...,Birth information:\n• Birth date: Circa from 1...,"Remark: Neka District, Paktika Province, Afgha...",Citizenship information:\n• Citizenship: Afgha...,Contact information: \n• Address: Pakistan \n,Remark: A leader of the Haqqani Network (TAe.0...,,,,,,,
3,.2274.87\n,2017/404 (OJ L63),AFG - Afghanistan,; Gul Agha Ishakzai ; Mullah Gul Agha ; Mu...,Birth information:\n• Birth date: Circa 1972 ...,Contact information: \n• Address: Pakistan \n...,Remark: Belongs to Ishaqzai tribe. INTERPOL-UN...,,,,,,,,,
4,.2291.0\n,2017/404 (OJ L63),AFG - Afghanistan,; Amir Abdullah Former Kandahar Province Dep...,Birth information:\n• Birth date: Circa 1972 ...,Citizenship information:\n• Citizenship: Afgha...,"Contact information: \n• Address: Pakistan, Ka...",Remark: Believed to be in Afghanistan/Pakistan...,,,,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
383,.3631.49\n,2017/396 (OJ L60),COD - Democractic Republic of Congo,; Mathieu Chui Ngudjolo Chief of Staff and f...,Birth information:\n• Birth date: 08/10/1970 ...,"Remark: Place of birth: Bunia, Ituri\nProvince...",Citizenship information:\n• Citizenship: Congo...,"Contact information: \n• Address: Congo, Democ...",Remark: Mathieu Chui Ngudjolo was the Chief of...,,,,,,,
384,.3632.14\n,2017/396 (OJ L60),COD - Democractic Republic of CongoPage 99 on ...,; Floribert Ngabu Njabu ; Floribert Njabu ...,Birth information:\n• Birth date: 23/05/1971 ...,Citizenship information:\n• Citizenship: Congo...,Remark: President of FNI\nEU,,,,,,,,,
385,.3633.76\n,2017/396 (OJ L60),COD - Democractic Republic of Congo,; 'Chairman' ; 'Papa Six' ; ģenerālis Nkun...,Birth information:\n• Birth date: 06/02/1967 ...,Citizenship information:\n• Citizenship: Congo...,,,,,,,,,,
386,.3634.41\n,2017/396 (OJ L60),COD - Democractic Republic of Congo,; Fred Irakeza FDLR-FOCA Colonel ; Félicie...,"Birth information:\n• Birth place: Rwanda, Kin...",Citizenship information:\n• Citizenship: Rwanda\n,"Contact information: \n• Address: Congo, Democ...",Remark: As of June 2016\nEU,,,,,,,,


In [12]:
dbn_df['Identity information'][1]

'; Sirajuddin Jallaloudine Haqqani   Na’ib Amir (Deputy Commander)   ; Khalifa   ; Saraj Haqani   ; Siraj Haqani   ; Serajuddin Haqani   ; Siraj Haqqani'

## Fuzzy
---


In [14]:
from thefuzz import fuzz, process 

s1 = 'pascal'

#x = input('gib dein scheiß hier ein')
x = 'Pascal'

fuzz.ratio(x, s1)



83

## Word2Vec
---

In [104]:
import gensim
from gensim.models import Word2Vec
from sklearn.decomposition import PCA
from nltk.tokenize import sent_tokenize, word_tokenize
from matplotlib import pyplot
 
#Preprocessing
data = dbn_df['Identity information'] = dbn_df['Identity information'].apply(lambda content: [x for x in content.split(';')]) #Translating 'Identity information' into list object
sentences_tokenized = []
for i in data:
    print(i)
    # s = i.lower
    # myPunc = '!"#%&$()*+-./:;,<=>?@[\\]^_`{|}~'
    # s = s.translate(s.maketrans(myPunc, ' '*len(myPunc)))
    # s = s.translate(s.maketrans(string.digits, ' '*len(string.digits)))
    # s = re.sub('\s+',' ',s)
    # print(s)
    # for sentence in s:    
    #     tokens = sentence.split(' ')
    #     sentences_tokenized.append(tokens)




['']
['', ' Sirajuddin Jallaloudine Haqqani   Na’ib Amir (Deputy Commander)   ', ' Khalifa   ', ' Saraj Haqani   ', ' Siraj Haqani   ', ' Serajuddin Haqani   ', ' Siraj Haqqani']
['', ' Nasiruddin Haqqani   A leader of the Haqqani Network, which operates out of North Waziristan in the FederallyAdministered  Tribal Areas of Pakistan.   ', ' Naseer Haqqani   ', ' Dr. Naseer Haqqani  Title: Dr.   ', ' Nassir Haqqani   ', ' Nashir Haqqani   ', ' Dr. Alim Ghair  Title: Dr.']
['', ' Gul Agha Ishakzai   ', ' Mullah Gul Agha   ', ' Mullah Gul Agha Akhund   ', ' Hidayatullah   ', ' Hayadatullah   ', ' Haji Hidayatullah']
['', ' Amir Abdullah   Former Kandahar Province Deputy Taliban Governor   ', ' Amir Abdullah Sahib']
['', ' Agha Jan Alizai   ', ' Haji Agha JanAlizai   ', ' Abdul Habib   ', ' Abdul Habib Alizai  Title: Haji   Has managed a drug trafficking network in Helmand Province, Afghanistan   ', ' Hajji Agha Jan   ', ' Agha Jan Alazai   ', ' Haji Loi Lala   ', ' Loi Agha']
['', ' Khalil

In [79]:
# data1 = []
# for i in dbn_df['Identity information']:
#     # temp = []
#     # for j in i:
#     #     temp.append(j.lower())
#     # data.append(temp)
#     data1.append(i.lower())




AttributeError: 'str' object has no attribute 'str'

In [72]:
# Creating Word2Vec
model = Word2Vec(
    sentences = data,
    #size = 50,
    window = 10,
    #iter = 20,
    
)




In [73]:
model.wv.key_to_index #Ausgabe aller Key-Vektor Paare in Word2Vec model


{'': 0}

In [24]:
ergebnisse = model.wv.most_similar('the', topn=5)

for y in ergebnisse:
    print(y)

KeyError: "Key 'the' not present in vocabulary"